In [ ]:
#Script to pre-process 2017-2019 air quality data to make ready for visualization
import os
import traceback

os.chdir('E:/Research/Processing Datasets/Air quality datasets')
count = 0
studyPeriodFiles = []
#Store filenames of files in 2017-2019
for ind, filename in enumerate(os.listdir(os.getcwd())):
    splits = filename.split("_")
    if splits[3] in ['2017', '2018', '2019']:
        studyPeriodFiles.append(filename)
print(len(studyPeriodFiles))

In [ ]:
import re
import time
import pandas as pd

start = time.process_time()
os.chdir('E:/Research/Processing Datasets/Air quality datasets')
airQualityData = []
count = 0
substart = time.process_time()
#Process each file
for ind, filename in enumerate(studyPeriodFiles[11:]):
    count+=1
    #To print status
    if count%1000 == 0:
        print('Procesed until ', count,'/',len(studyPeriodFiles),' in ',time.process_time() - substart)
        substart = time.process_time()
    try:
        splits = filename.split("_")
        try:
            df = pd.read_csv(filename, error_bad_lines=False, encoding = "utf-16")
        except UnicodeError:
            df = pd.read_csv(filename, error_bad_lines=False)
        
        #Filter invalid data
        stationCodes = [code for code in df['AirQualityStationEoICode'].unique() if type(code)==str]
        pollutants = [pol for pol in df['AirPollutant'].unique() if type(pol)==str and not pol.replace('.', '').isdigit()]
        avgTimes = [time for time in df['AveragingTime'].unique() if type(time)==str]
        UOMs = [uni for uni in df['UnitOfMeasurement'].unique() if type(uni) == str and re.search('[a-zA-Z]', uni)]
        
        #Process records
        if len(stationCodes) > 0:
            temp = df[(df['AirQualityStationEoICode'].isin(stationCodes)) & (df['AirPollutant'].isin(pollutants)) & (df['AveragingTime'].isin(avgTimes)) & (df['UnitOfMeasurement'].isin(UOMs))]
            temp['DateBegin'] = pd.to_datetime(temp['DatetimeBegin']).dt.date
            temp['Concentration'] = temp['Concentration'].astype(float)
                
            for avgTime in avgTimes:
                #Find daily mean of hourly reading
                groupedDf = temp[temp['AveragingTime'] == avgTime].groupby(['Countrycode','AirQualityStationEoICode', 'DateBegin', 'AirPollutant', 'UnitOfMeasurement','AveragingTime'], as_index=False).agg({'Concentration': 'mean'})
                #airQualityData.extend(groupedDf.to_dict('records'))
                #Append after the file was created
                #Append to csv file
                groupedDf.to_csv('D:/AirQualityDataset/AirQualityStudyPeriodDataNew.csv', mode='a', header=False, index = False, columns=['Countrycode', 'AirPollutant', 'AirQualityStationEoICode', 'AveragingTime','DateBegin','Concentration', 'UnitOfMeasurement'])
                
    except Exception:
        print('Exception while processing file ', ind, ' - ', filename)
        print(traceback.format_exc())
        continue

#Create new file using first 10 files
#pd.DataFrame(airQualityData).to_csv('D:/AirQualityDataset/AirQualityStudyPeriodDataNew.csv', index = False, columns=['Countrycode', 'AirPollutant', 'AirQualityStationEoICode', 'AveragingTime','DateBegin','Concentration', 'UnitOfMeasurement'])
print("Completed processing in ", time.process_time() - start)

In [ ]:
#Script to produce formatted data for visualizations
import math
import os
import pandas as pd
import time
import numpy as np

#Read the data produced by above script(Around 500MB)
df = pd.read_csv('AirQualityStudyPeriodDataNew.csv')
df = df[df['AirPollutant'].isin(['CO', 'SO2', 'C6H6', 'PM10', 'PM2.5', 'O3', 'NO2'])]
#extract year
df['Year'] = pd.to_datetime(df['DateBegin']).dt.year

#Create boolean for exceeding EU standard limits for each pollutant
df['Concentration'] = pd.to_numeric(df['Concentration'],errors='coerce')
df['ExceededLimitCO'] = np.where(((df.AirPollutant == 'CO') & (df.AveragingTime.isin(['hour', 'day', 'var', 'fortnight'])) & (df.Concentration > 7.0)), True, False)
df['ExceededLimitC6H6'] = np.where(((df.AirPollutant == 'C6H6') & (df.AveragingTime == 'year') & (df.Concentration > 3.5)), True, False)
df['ExceededLimitSO2'] = np.where(((df.AirPollutant == 'SO2') & (df.AveragingTime.isin(['hour', 'day', 'var', 'fortnight']))& (df.Concentration > 75.0)), True, False)
df['ExceededLimitPM2'] = np.where(((df.AirPollutant == 'PM2.5') & (df.AveragingTime.isin(['hour', 'var', 'day', 'fortnight', 'year'])) & (df.Concentration > 25.0)), True, False)
df['ExceededLimitPM10'] = np.where(((df.AirPollutant == 'PM10') & (df.AveragingTime.isin(['hour', 'var', 'day', 'fortnight'])) & (df.Concentration > 50.0)), True, np.where(((df.AirPollutant == 'PM10') & (df.AveragingTime =='year') & (df.Concentration > 40.0)), True, False))
df['ExceededLimitO3'] = np.where(((df.AirPollutant == 'O3') & (df.AveragingTime.isin(['hour', 'var', 'day', 'fortnight'])) & (df.Concentration > 120.0)), True, False)
df['ExceededLimitNO2'] = np.where(((df.AirPollutant == 'NO2') & (df.AveragingTime.isin(['hour', 'var', 'day', 'fortnight'])) & (df.Concentration > 200.0)), True, np.where(((df.AirPollutant == 'NO2') & (df.AveragingTime == 'year') & (df.Concentration > 40.0)), True, False))

#Finally create one boolean for all the pollutants
df['Exceeded'] = np.where(((df.ExceededLimitCO == True) | (df.ExceededLimitC6H6 == True) | (df.ExceededLimitSO2 == True) | (df.ExceededLimitPM2 == True) | (df.ExceededLimitPM10 == True) | (df.ExceededLimitO3 == True) | (df.ExceededLimitNO2 == True) ), True, False )
                                   
df.head()

In [ ]:
#Extract counts of times the limit has been exceeded for each pollutant
exceedingPollutantsDF = df.groupby(['AirPollutant', 'Year']).apply(lambda x: len(x['Exceeded'] == True)).reset_index()
exceedingDF.to_csv("pollutantExceedingData.csv", index = False)

In [ ]:
#Extract counts of times the limit has been exceeded for each country
exceedingCountriesDF = df.groupby(['Countrycode']).apply(lambda x: len(x['Exceeded'] == True)).reset_index()
exceedingCountriesDF.to_csv("countryExceedingData.csv", index = False)

In [ ]:
#Script to merge with Air Quality Locations dataset to add geo-co-ordinate values

#Read locations data
stations = pd.read_csv('AirQualityStations.csv')
#Extract needed columns, drop na and duplicates
stations = stations[['EoICode', 'CountryOrTerritory','Latitude', 'Longitude']].dropna(subset=['Latitude', 'Longitude']).drop_duplicates('EoICode')
#rename common column to make ready for merge
stations.rename(columns={'EoICode': 'AirQualityStationEoICode'}, inplace=True)
#filter out stations that do not have location co-ordinates
locationBased = df[df['AirQualityStationEoICode'].isin(stations['AirQualityStationEoICode'].values)]
#Merge datasets
locationBased = pd.merge(locationBased, stations, on='AirQualityStationEoICode')

#Extract counts of times the limit has been exceeded for each station
exceedingstations =locationBased.groupby(['CountryOrTerritory','AirQualityStationEoICode', 'Latitude', 'Longitude']).apply(lambda x: len(x['Exceeded'] == True)).reset_index()
exceedingstations.rename(columns={0:'exceedCount'}, inplace=True )
#Store in file
exceedingstations.to_csv('exceedingstations.csv', index = False)